In [1]:
import torch
from datatypes import *
from train_dm import *
import os
import json
import config
from collections import deque

In [2]:
path='_outputs/run_262/'
outpath=path+'/eval.txt'

In [3]:
if os.path.exists(outpath):
    raise ValueError('Output file already exists')

In [4]:
with open(os.path.join(path, 'args.json'), 'r') as f:
    args = json.load(f)

In [5]:
sorted(os.listdir(path))

['H.pt',
 'args.json',
 'block_t-0.pt',
 'block_t-1.pt',
 'block_t-10.pt',
 'block_t-11.pt',
 'block_t-12.pt',
 'block_t-13.pt',
 'block_t-14.pt',
 'block_t-15.pt',
 'block_t-16.pt',
 'block_t-17.pt',
 'block_t-18.pt',
 'block_t-19.pt',
 'block_t-2.pt',
 'block_t-20.pt',
 'block_t-21.pt',
 'block_t-22.pt',
 'block_t-23.pt',
 'block_t-24.pt',
 'block_t-3.pt',
 'block_t-4.pt',
 'block_t-5.pt',
 'block_t-6.pt',
 'block_t-7.pt',
 'block_t-8.pt',
 'block_t-9.pt',
 'params_t-0.pt',
 'params_t-1.pt',
 'params_t-10.pt',
 'params_t-11.pt',
 'params_t-12.pt',
 'params_t-13.pt',
 'params_t-14.pt',
 'params_t-15.pt',
 'params_t-16.pt',
 'params_t-17.pt',
 'params_t-18.pt',
 'params_t-19.pt',
 'params_t-2.pt',
 'params_t-20.pt',
 'params_t-21.pt',
 'params_t-22.pt',
 'params_t-23.pt',
 'params_t-24.pt',
 'params_t-3.pt',
 'params_t-4.pt',
 'params_t-5.pt',
 'params_t-6.pt',
 'params_t-7.pt',
 'params_t-8.pt',
 'params_t-9.pt',
 'rho_t-0.pt',
 'rho_t-1.pt',
 'rho_t-10.pt',
 'rho_t-11.pt',
 'rho_t-12

In [6]:
config

<module 'config' from '/home/nilin/diffcircuit/config.py'>

In [7]:
blockfiles=[f for f in os.listdir(path) if 'block' in f]
blockfiles.sort(key=lambda s: int(s.split('-')[1].split('.')[0]))
blockfiles.reverse()
#blockfiles=deque(blockfiles)

In [8]:
blockfiles

['block_t-24.pt',
 'block_t-23.pt',
 'block_t-22.pt',
 'block_t-21.pt',
 'block_t-20.pt',
 'block_t-19.pt',
 'block_t-18.pt',
 'block_t-17.pt',
 'block_t-16.pt',
 'block_t-15.pt',
 'block_t-14.pt',
 'block_t-13.pt',
 'block_t-12.pt',
 'block_t-11.pt',
 'block_t-10.pt',
 'block_t-9.pt',
 'block_t-8.pt',
 'block_t-7.pt',
 'block_t-6.pt',
 'block_t-5.pt',
 'block_t-4.pt',
 'block_t-3.pt',
 'block_t-2.pt',
 'block_t-1.pt',
 'block_t-0.pt']

In [9]:
n=args['n']
H=torch.load(path+'/H.pt')
psi_target = groundstate(H, n)

I = torch.eye(len(psi_target), dtype=tcomplex, device=config.device) / len(psi_target)

#print('requires_grad',rho.requires_grad)

blocks={i:None for i in range(len(blockfiles))}

def apply(blockfiles,n_superblocks,rho):
    for _ in range(n_superblocks):
        for i,bf in enumerate(blockfiles):
            blocks[i]=torch.load(path+'/'+bf)
            rho=blocks[i].apply_to_density_matrix(rho)
            del blocks[i]
            rho=rho.detach()
            #print(rho.shape)
            #print(torch.cuda.memory_summary())

        yield cdot(psi_target, rho @ psi_target).real

In [10]:
def retrieve(tensor):
    return tensor.detach().cpu().numpy()

for d in range(1,len(blockfiles)):
    for s,out in enumerate(apply(blockfiles[-d:],10,I)):
        print('d',d,'superblock',s,'value',retrieve(out))

        with open(outpath, "a") as f:
            f.write(f"d {d} superblocks {s} value {retrieve(out)}\n")

    print()
        #print(out)I

d 1 superblock 0 value 0.0039058635
d 1 superblock 1 value 0.0039502657
d 1 superblock 2 value 0.0039943107
d 1 superblock 3 value 0.0040382263
d 1 superblock 4 value 0.0040823855
d 1 superblock 5 value 0.004126977
d 1 superblock 6 value 0.0041720634
d 1 superblock 7 value 0.004217682
d 1 superblock 8 value 0.0042638243
d 1 superblock 9 value 0.0043104803

d 2 superblock 0 value 0.0078087775
d 2 superblock 1 value 0.015180653
d 2 superblock 2 value 0.028649641
d 2 superblock 3 value 0.053135518
d 2 superblock 4 value 0.09644802
d 2 superblock 5 value 0.16624592
d 2 superblock 6 value 0.22764893
d 2 superblock 7 value 0.2847613
d 2 superblock 8 value 0.3261035
d 2 superblock 9 value 0.36988726

d 3 superblock 0 value 0.015610464
d 3 superblock 1 value 0.061759416
d 3 superblock 2 value 0.11442203
d 3 superblock 3 value 0.16259418
d 3 superblock 4 value 0.20873252
d 3 superblock 5 value 0.26059607
d 3 superblock 6 value 0.31331238
d 3 superblock 7 value 0.3599042
d 3 superblock 8 value 0